In [6]:
#Library imports
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

#Makes Dataframes fit into window
pd.set_option('display.max_rows', 500)

#Directory location of files
string_loc = "/Users/Nicole/Documents/BAC Insight Team/Data/"

In [7]:
#Initialized array to be filled with every day's data file
days = []

#Walks through file directory looking for files in 2017
#Adds vetted data files into 'days' array
for root, dirs, file in os.walk(string_loc):
    #if len(file) > 50:
        for i in file:
            if i.endswith(".txt"):
                days.append(i)

#Sorts the array chronologically
days.sort()

In [8]:
days

['20170701.txt',
 '20170702.txt',
 '20170703.txt',
 '20170704.txt',
 '20170705.txt',
 '20170706.txt',
 '20170707.txt',
 '20170708.txt',
 '20170709.txt',
 '20170710.txt',
 '20170711.txt',
 '20170712.txt',
 '20170713.txt',
 '20170714.txt',
 '20170715.txt',
 '20170716.txt',
 '20170717.txt',
 '20170718.txt',
 '20170719.txt',
 '20170720.txt',
 '20170721.txt',
 '20170722.txt',
 '20170723.txt',
 '20170724.txt',
 '20170725.txt',
 '20170726.txt',
 '20170727.txt',
 '20170728.txt',
 '20170729.txt',
 '20170730.txt',
 '20170731.txt',
 '20170801.txt',
 '20170802.txt',
 '20170803.txt',
 '20170804.txt',
 '20170805.txt',
 '20170806.txt',
 '20170807.txt',
 '20170808.txt',
 '20170809.txt',
 '20170810.txt',
 '20170811.txt',
 '20170812.txt',
 '20170813.txt',
 '20170814.txt',
 '20170815.txt',
 '20170816.txt',
 '20170817.txt',
 '20170818.txt',
 '20170819.txt',
 '20170820.txt',
 '20170821.txt',
 '20170822.txt',
 '20170823.txt',
 '20170824.txt',
 '20170825.txt',
 '20170826.txt',
 '20170827.txt',
 '20170828.txt

In [9]:
#Initialized array of dataframes
#'dataframes' contains dataframe of every day's data
#'adj_dataframes' contains modified dataframe of every day's data
dataframes = []
adj_dataframes = []

In [10]:
#Reads in every day's data in the 'days' array
#Appends the created Dataframe into the 'dataframes' array (the array holds Dataframe objects)
#(The 'dataframes' array maintains chronological order)
for day in days:
    df = pd.read_csv(string_loc + "/" + day[0:4] + "/" + day[4:6] + "/" + day, index_col = None, sep = ";")
    dataframes.append(df)

In [11]:
#Accepts a row's 'symbol_id' value to return the exchange
def exchange(symbol_id):
    index = symbol_id.find("SPOT")
    exchange = symbol_id[:index-1]
    return (exchange)

#Accepts a row's 'symbol_id' value to return the pair
def Pair(symbol_id):
    index = symbol_id.find("SPOT")
    pair = symbol_id[index+5:]
    return(pair)

In [12]:
#For every object in the 'dataframes' array, resets the index numbering to 0,1,2,3...
for df in dataframes:
    df.reset_index(drop=True, inplace=True)

In [13]:
#Inititalized arrays that hold a given Dataframe's pairs and exchange values in ARRAY FORM
#These arrays will hold other arrays
pair = []
exchanges = []

#Goes through every dataframe, finding each row's exchange and pair and storing them into a temporary array
#The calculated exchange and pair are appended into 'subPair' and 'subExchanges'
for d in dataframes:
    subPair = []
    subExchanges = []
    for row in d.itertuples():
        subPair.append(Pair(row[1]))
        subExchanges.append(exchange(row[1]))
        
    #After iterating through a dataframe, the temporary arrays are appended into 'pair' and 'exchanges'
    pair.append(subPair)
    exchanges.append(subExchanges)

In [14]:
#Goes through every dataframe and adds 'pair' and 'exchange' columns
for pairs, exchange, df in zip(pair, exchanges, dataframes):
    df["pair"] = pairs
    df["exchange"] = exchange

In [15]:
for df in dataframes:
    
    #Initializes two arrays that will hold a dataframe's start and end times for each row 
    start_times = []
    end_times = []
    
    #Sorts each day's dataframe in terms of 'symbol_id' and starting time
    df.sort_values(by = (["symbol_id", "time_period_start"]), inplace = True)
    
    #Converts the time columns into 'datetime' objects
    df.time_period_start = pd.to_datetime(df.time_period_start)
    df.time_period_end = pd.to_datetime(df.time_period_end)
    
    dt_start_times = list(df.time_period_start.values)
    dt_end_times = list(df.time_period_end.values)
    
    #Extracts just the hour in the 'start_time' column and appends it into the 'start_times' array
    for f in range(len(dt_start_times)):
        string = str(dt_start_times[f])
        substring = string[-18:-10]
        start_times.append(substring)
        
    #Extracts just the hour in the 'end_times' column and appends it into the 'end_times' array
    for e in range(len(dt_end_times)):
        string = str(dt_end_times[e])
        substring = string[-18:-10]
        end_times.append(substring)
        
    #Initializes arrays that will hold the values to be converted into a new Dataframe.    
    symbol_ids = np.array(df.symbol_id.values)
    Open = np.array(df.px_open.values)
    close = np.array(df.px_close.values)
    transactions = np.array(df.sx_cnt.values)
    base_volume = np.array(df.sx_sum.values)
    fiat_volume = base_volume * close
    pair = np.array(df.pair.values)
    exchanges = np.array(df.exchange.values)
 
    #Arrays converted into dictionary
    dictionary = {"name" : symbol_ids,
                   "exchange" : exchanges,
                   "pair" : pair,
                   "total_base_volume" : np.nan,
                   "total_dollar_volume" : np.nan,
                   "WAPP" : np.nan,
                   "start_time" : start_times,
                   "end_times" : end_times,
                   "open" : Open,
                   "close" : close,
                   "transactions" : transactions,
                   "base_volume" : base_volume,
                   "dollar_volume" : fiat_volume}
    
    #Dictionary converted Dataframe and vetted for 'BTC_USD' and 'BTC_USDT' only
    df2 = pd.DataFrame(dictionary)
    df2 = df2[((df2.pair.str.startswith("BTC_USD")))]
    df2.reset_index(drop=True, inplace=True)
    
    #Appends the newly creted Dataframe into the 'adj_dataframes' array
    adj_dataframes.append(df2)

In [16]:
# create dataframe with daily close price at time 00:00:00
close_df = []
for df in adj_dataframes:
    close_time = (df['end_times'] == '00:00:00')
    temp_df = df[close_time]
    close_df.append(temp_df)
    

In [17]:
close_df[1]

,WAPP,base_volume,close,dollar_volume,end_times,exchange,name,open,pair,start_time,total_base_volume,total_dollar_volume,transactions
23,NaN,761.881959,2445.100000,1.862878e+06,00:00:00,BITFINEX,BITFINEX_SPOT_BTC_USD,2437.200000,BTC_USD,23:00:00,NaN,NaN,1436
47,NaN,487.493214,2504.370000,1.220863e+06,00:00:00,BITSTAMP,BITSTAMP_SPOT_BTC_USD,2509.990000,BTC_USD,23:00:00,NaN,NaN,731
71,NaN,64.345557,2438.000000,1.568745e+05,00:00:00,BITTREX,BITTREX_SPOT_BTC_USDT,2422.000000,BTC_USDT,23:00:00,NaN,NaN,236
95,NaN,80.788881,2422.560000,1.957159e+05,00:00:00,BTC-E,BTC-E_SPOT_BTC_USD,2414.601000,BTC_USD,23:00:00,NaN,NaN,725
116,NaN,21.640000,2508.390000,5.428156e+04,00:00:00,BTCX,BTCX_SPOT_BTC_USD,2517.580000,BTC_USD,23:00:00,NaN,NaN,51
140,NaN,463.281810,2516.660000,1.165923e+06,00:00:00,COINBASE,COINBASE_SPOT_BTC_USD,2517.010000,BTC_USD,23:00:00,NaN,NaN,1679
170,NaN,10.185768,2540.000000,2.587185e+04,00:00:00,EXMO,EXMO_SPOT_BTC_USD,2534.990000,BTC_USD,23:00:00,NaN,NaN,59
178,NaN,150.250000,2558.800000,3.844597e+05,00:00:00,GATECOIN,GATECOIN_SPOT_BTC_USD,2558.700000,BTC_USD,23:00:00,NaN,NaN,72
202,NaN,17.190000,2438.960000,4.192572e+04,00:00:00,HITBTC,HITBTC_SPOT_BTC_USDT,2437.750000,BTC_USDT,23:00:00,NaN,NaN,105
221,NaN,0.007100,2515.560000,1.786048e+01,00:00:00,ITBIT,ITBIT_SPOT_BTC_USD,2515.560000,BTC_USD,23:00:00,NaN,NaN,1


In [18]:
# subset 'BTC_USD' and 'BTC_USDT' pair transactions into two separate dataframes
usd_all_df = []
t_all_df = []
for df in close_df:
    temp_usd_df = df[df.pair == 'BTC_USD']
    usd_all_df.append(temp_usd_df)
    
    temp_t_df = df[df.pair == 'BTC_USDT']
    t_all_df.append(temp_t_df)

In [21]:
t_all_df[100]

,WAPP,base_volume,close,dollar_volume,end_times,exchange,name,open,pair,start_time,total_base_volume,total_dollar_volume,transactions
23,NaN,24.589105,4786.950000,117706.816180,00:00:00,BINANCE,BINANCE_SPOT_BTC_USDT,4760.50,BTC_USDT,23:00:00,NaN,NaN,432
106,NaN,147.029608,4775.700000,702169.299164,00:00:00,BITTREX,BITTREX_SPOT_BTC_USDT,4750.00,BTC_USDT,23:00:00,NaN,NaN,814
245,NaN,0.856414,4786.500000,4099.223840,00:00:00,EXMO,EXMO_SPOT_BTC_USDT,4752.00,BTC_USDT,23:00:00,NaN,NaN,37
315,NaN,21.770000,4782.170000,104107.840900,00:00:00,HITBTC,HITBTC_SPOT_BTC_USDT,4753.09,BTC_USDT,23:00:00,NaN,NaN,598
391,NaN,3.441445,4822.916041,16597.800344,00:00:00,LIQUI,LIQUI_SPOT_BTC_USDT,4802.00,BTC_USDT,23:00:00,NaN,NaN,286
439,NaN,166.557829,4787.657550,797421.846566,00:00:00,POLONIEX,POLONIEX_SPOT_BTC_USDT,4750.00,BTC_USDT,23:00:00,NaN,NaN,843


In [22]:
# only compare volume on the top 3 exchanges for each category
t_df = []
for df in t_all_df:
    exchange = (df.exchange == 'BINANCE') | (df.exchange == 'OKEX') | (df.exchange == 'HUOBIPRO')
    subset = df[exchange]
    t_df.append(subset)

In [23]:
t_df[50]

,WAPP,base_volume,close,dollar_volume,end_times,exchange,name,open,pair,start_time,total_base_volume,total_dollar_volume,transactions
23,NaN,69.475467,4086.29,283896.906047,00:00:00,BINANCE,BINANCE_SPOT_BTC_USDT,4151.14,BTC_USDT,23:00:00,NaN,NaN,243


In [32]:
usd_df[1].exchange.value_counts()

BITFINEX      1
BITSTAMP      1
OKCOIN_USD    1
ITBIT         1
EXMO          1
COINBASE      1
GATECOIN      1
BTC-E         1
BTCX          1
KRAKEN        1
LAKEBTC       1
Name: exchange, dtype: int64

In [204]:
# create list of dataframes containing the row with lowest close price for each day for BTC to USD
low_usd = []
for df in usd_df:
    low_usd_close = min(df.close)
    low_usd_row = df[df.close == low_usd_close]
    low_usd.append(low_usd_row)

In [205]:
# create list of exchanges with the lowest close each day for BTC to USD
low_usd_list = []
for df in low_usd:
    low_usd_list.append(df.exchange.values[0])

In [24]:
# create list of dataframes containing the row with lowest close price for each day for BTC to Tether
low_t = []
for df in t_df:
    try:
        low_t_close = min(df.close)
        low_t_row = df[df.close == low_t_close]
        low_t.append(low_t_row)
    except:
        low_t.append('None')

In [25]:
# create list of exchanges with the lowest close each day for BTC to Tether
low_t_list = []
for df in low_t:
    try:
        low_t_list.append(df.exchange.values[0])
    except:
        low_t_list.append('None')

In [26]:
low_t_list

['None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 'BINANCE',
 

In [221]:
low_usd_list

['BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'OKCOIN_USD',
 'OKCOIN_USD',
 'BTC-E',
 'OKCOIN_USD',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'OKCOIN_USD',
 'BTC-E',
 'OKCOIN_USD',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'OKCOIN_USD',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'OKCOIN_USD',
 'BTC-E',
 'OKCOIN_USD',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'KRAKEN',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'BTC-E',
 'KRAKEN',
 'KRAKEN',
 'KRAKEN',
 'ITBIT',
 'KRAKEN',
 'KRAKEN',
 'KRAKEN',
 'KRAKEN',
 'KRAKEN',
 'KRAKEN',
 'ITBIT',
 'KRAKEN',
 'BTCX',
 'BITSTAMP',
 'BITSTAMP',
 'BITSTAMP',
 'BTC-E',
 'BITSTAMP',
 'EXMO',
 'EXMO',


In [216]:
# Exchange that had the lowest close price most frequently for tether
max(low_t_list, key = low_t_list.count)

'HITBTC'

In [217]:
# Exchange that had the lowest close price most frequently for USD
max(low_usd_list, key = low_t_list.count)

'EXMO'

In [236]:
low_usd_df = pd.DataFrame(low_usd_list,columns=['Exchange'])

In [237]:
low_usd_df.head()

,Exchange
0,BTC-E
1,BTC-E
2,BTC-E
3,BTC-E
4,BTC-E


In [240]:
low_usd_df.Exchange.value_counts()

BTC-E             114
EXMO               79
COINFLOOR          37
BITFINEX           31
OKCOIN_USD         30
KRAKEN             15
BITSTAMP           14
BTCX               10
ITBIT               8
GATECOIN            6
QUADRIGACX          6
DSX                 4
BTCC                4
COINBASE            3
BITKONAN            2
THEROCKTRADING      2
Name: Exchange, dtype: int64

In [242]:
low_t_df = pd.DataFrame(low_t_list,columns=['Exchange'])
low_t_df.Exchange.value_counts()

HITBTC       89
BITTREX      63
POLONIEX     59
CRYPTOPIA    49
BINANCE      37
LIQUI        24
EXMO         21
None         17
OKEX          3
TIDEX         3
Name: Exchange, dtype: int64

In [219]:
binance_low = []
for i in low_t_list:
    if i == 'BINANCE':
        binance_low.append(True)
    else:
        binance_low.append(False)

In [ ]:
binance_low